In [ ]:
import hvplot.xarray
import numpy as np
import scipy.stats
import xarray as xr

import climepi
import climepi.climdata as climdata
import climepi.epimod as epimod

In [ ]:
ds_clim = climdata.get_example_dataset("isimip_london")
ds_clim["time"].attrs["long_name"] = "Time"
climepi_modes = ds_clim.climepi.modes
ds_clim = (
    ds_clim.drop_vars(["lon_bnds", "lat_bnds"])
    .squeeze()
    .sel({"scenario": ["ssp126", "ssp370", "ssp585"]})
)
ds_clim.climepi.modes = climepi_modes

In [ ]:
ds_clim_ym = ds_clim.climepi.yearly_average()
ds_clim_ym.climepi.plot_time_series("temperature", by=["scenario"])

In [ ]:
ecol_niche_model = epimod.ecolniche.get_kaye_model()
ds_clim.epimod.model = ecol_niche_model
ds_epi = ds_clim.epimod.run_model()
ds_months_suitable = ds_epi.epimod.months_suitable()
ds_months_suitable.climepi.plot_time_series(by=["scenario"])

In [ ]:
ds_ym = xr.merge([ds_clim_ym, ds_months_suitable])
ds_ym

In [ ]:
ds_ym_stat = ds_ym.climepi.ensemble_stats(
    conf_level=90,
    estimate_internal_variability=True,
)
ds_ym_stat

In [ ]:
data_var_plot = "months_suitable"

In [ ]:
p1 = ds_ym_stat.sel(ensemble_stat="mean").climepi.plot_time_series(
    data_var_plot, by=["scenario"]
)
p2 = ds_ym.climepi.plot_time_series(data_var_plot, by=["scenario"])
p1 * p2

In [ ]:
ds_ym.climepi.plot_var_decomp(
    data_var=data_var_plot, estimate_internal_variability=True, fraction=True
)

In [ ]:
ds_ym.climepi.plot_ci_plume(
    data_var=data_var_plot,
    conf_level=90,
    estimate_internal_variability=True,
    alpha=0.6,
)

In [ ]:
scenario_ex = "ssp370"
model_ex = "ipsl-cm6a-lr"
ds_ym.sel(scenario=scenario_ex).climepi.plot_ci_plume(
    data_var=data_var_plot, conf_level=90, estimate_internal_variability=True
) * ds_ym.sel(scenario=scenario_ex).climepi.plot_time_series(data_var=data_var_plot)

In [ ]:
ds_ym.sel(scenario=scenario_ex).climepi.plot_ci_plume(
    data_var=data_var_plot,
    conf_level=90,
    estimate_internal_variability=True,
) * ds_ym.sel(scenario=scenario_ex).climepi.plot_time_series(
    data_var=data_var_plot, by="model"
)

In [ ]:
ds_ym.sel(model=model_ex).climepi.plot_ci_plume(
    data_var=data_var_plot, conf_level=90, estimate_internal_variability=True
) * ds_ym.sel(model=model_ex).climepi.plot_time_series(
    data_var=data_var_plot, by="scenario"
)

In [ ]:
ds_ym.sel(scenario=scenario_ex, model=model_ex).climepi.plot_ci_plume(
    data_var=data_var_plot, conf_level=90, estimate_internal_variability=True
) * ds_ym.sel(scenario=scenario_ex, model=model_ex).climepi.plot_time_series(
    data_var=data_var_plot
)